In [1]:
import pandas as pd
import surprise
import numpy as np
import os.path
import csv
import operator

In [2]:
# 영화 300개 자료 저장
# User 수는 1000명
data = {}
movie_id = 0
user_name = 0
rating = 0
user_list = []


with open('combined_data_1.txt', 'r') as f:
    for line in f:
        if(line.find(":") != -1):
            continue
        line_split = line.split(',')
        user_name = int(line_split[0])
        if user_name in user_list:
            continue
        else:
            user_list.append(user_name)
        if len(user_list) > 1000:
            break
        
#print(user_list)

with open('combined_data_1.txt', 'r') as f:
    for line in f:
        if(line.find(":") != -1):
            movie_id = int(line.replace(":","")) - 1
            if(movie_id >= 300): break
            data[movie_id] = {}
        else:
            line_split = line.split(',')
            user_name = int(line_split[0])
            rating = int(line_split[1])
            if user_name in user_list:
                data[movie_id][user_name] = rating
#print(data)
#print(pd.DataFrame(data))

In [30]:
### 영화, 사용자, rating 정보를 2차원 배열에 저장
### 1000 x 300 영화, 사용자 2차원 배열

matrix = [[np.nan for col in range(300)] for row in range(1000)]

for i in range(1000):
    for j in range(300):
        if(user_list[i] in data[j]):
            matrix[i][j] = data[j][user_list[i]]
        
matrix = np.array(matrix)

matrix[:5]

array([[ 3., nan, nan, ..., nan,  3., nan],
       [ 5., nan, nan, ..., nan, nan, nan],
       [ 4., nan, nan, ..., nan, nan, nan],
       [ 4., nan, nan, ..., nan,  4., nan],
       [ 3., nan, nan, ..., nan, nan, nan]])

In [31]:
def EUC(a):
    a = np.array(pd.DataFrame(a).fillna(False))
    NumUsers = np.size(a, axis = 0)
    Sim = np.full((NumUsers, NumUsers), 0.0)
    for u in range(0, NumUsers):
        for v in range(u, NumUsers):
            tmp = np.sum(np.square(a[u, ] - a[v, ]))
            Sim[u,v] = np.sqrt(tmp)
            Sim[v,u] = Sim[u,v]
    return Sim

pd.DataFrame(EUC(matrix))

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,0.000000,18.220867,21.142375,17.691806,16.733201,18.220867,17.435596,18.357560,17.435596,18.110770,...,18.138357,18.867962,18.330303,19.798990,19.131126,18.466185,19.364917,19.235384,19.493589,18.248288
1,18.220867,0.000000,12.206556,16.703293,15.033296,10.295630,5.099020,13.892444,8.717798,2.000000,...,6.855655,10.488088,7.348469,10.488088,14.422205,5.385165,17.464249,10.862780,9.591663,11.269428
2,21.142375,12.206556,0.000000,19.798990,17.521415,15.066519,11.958261,17.720045,13.820275,12.206556,...,12.961481,15.779734,13.892444,15.779734,17.748239,12.961481,20.832667,16.031220,15.198684,16.309506
3,17.691806,16.703293,19.798990,0.000000,17.972201,16.643317,16.217275,15.427249,16.522712,16.703293,...,16.733201,19.000000,16.822604,18.248288,19.519221,17.262677,21.863211,17.521415,18.357560,19.899749
4,16.733201,15.033296,17.521415,17.972201,0.000000,15.165751,14.422205,16.093477,14.071247,14.899664,...,15.132746,17.776389,14.491377,17.204651,15.427249,15.329710,20.760539,16.852300,16.370706,15.066519
5,18.220867,10.295630,15.066519,16.643317,15.165751,0.000000,11.313708,15.588457,11.401754,10.099505,...,11.532563,12.083046,10.000000,13.341664,14.352700,10.723805,16.583124,12.649111,13.341664,14.035669
6,17.435596,5.099020,11.958261,16.217275,14.422205,11.313708,0.000000,13.228757,10.000000,5.099020,...,5.744563,11.224972,8.366600,11.224972,14.966630,6.708204,17.916473,11.575837,10.392305,11.958261
7,18.357560,13.892444,17.720045,15.427249,16.093477,15.588457,13.228757,0.000000,14.730920,13.747727,...,14.000000,16.822604,13.674794,16.822604,17.521415,14.212670,20.542639,16.093477,16.278821,16.733201
8,17.435596,8.717798,13.820275,16.522712,14.071247,11.401754,10.000000,14.730920,0.000000,8.717798,...,10.630146,13.266499,9.486833,12.806248,12.727922,9.746794,17.058722,12.649111,11.832160,13.892444
9,18.110770,2.000000,12.206556,16.703293,14.899664,10.099505,5.099020,13.747727,8.717798,0.000000,...,5.567764,9.695360,6.164414,9.695360,13.856406,3.605551,17.000000,10.099505,8.717798,10.535654


In [35]:
def COS(a):
    a = np.array(pd.DataFrame(a).fillna(0))
    NumUsers = np.size(a, axis = 0)
    Sim = np.full((NumUsers, NumUsers), 0.0)
    print("COS calculation start!")
    for u in range(0, NumUsers):
        arridx_u = np.where(a[u, ] == 0)
        for v in range(u, NumUsers):
            arridx_v = np.where(a[v, ] == 0)
            arridx = np.unique(np.concatenate((arridx_u, arridx_v), axis = None))
            
            U = np.delete(a[u, ], arridx)
            V = np.delete(a[v, ], arridx)
            
            u_sum = np.linalg.norm(U)
            v_sum = np.linalg.norm(V)
            uv_dot = np.dot(a[u], a[v])
            
            if(u_sum == 0 or v_sum == 0):
                Sim[u,v] = 0
            else:
                Sim[u,v] = uv_dot / (u_sum * v_sum)
            Sim[v,u] = Sim[u,v]
    
    print("COS calculation end!")
    return Sim

pd.DataFrame(COS(matrix))

COS calculation start!
COS calculation end!


,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,1.000000,1.0,0.926092,0.927496,0.907854,0.990332,0.990992,0.973117,0.992795,1.0,...,1.000000,0.991837,1.000000,0.980581,0.975544,0.0,0.977078,0.962250,0.894427,0.835086
1,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
2,0.926092,1.0,1.000000,0.812404,0.971797,1.000000,0.968277,0.960000,1.000000,1.0,...,1.000000,0.000000,0.000000,0.000000,1.000000,0.0,1.000000,0.000000,0.000000,0.000000
3,0.927496,1.0,0.812404,1.000000,0.983041,0.992159,0.993884,0.977154,0.989949,1.0,...,0.998460,1.000000,1.000000,0.854199,1.000000,0.0,0.947758,0.990148,1.000000,0.000000
4,0.907854,1.0,0.971797,0.983041,1.000000,0.960392,0.999512,0.972588,0.956681,1.0,...,1.000000,0.000000,1.000000,1.000000,0.995702,0.0,0.975610,1.000000,1.000000,0.977467
5,0.990332,1.0,1.000000,0.992159,0.960392,1.000000,1.000000,0.957024,0.964209,1.0,...,0.000000,1.000000,1.000000,1.000000,0.991837,0.0,0.975100,0.983870,0.000000,1.000000
6,0.990992,1.0,0.968277,0.993884,0.999512,1.000000,1.000000,0.999512,1.000000,1.0,...,1.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
7,0.973117,1.0,0.960000,0.977154,0.972588,0.957024,0.999512,1.000000,0.964209,1.0,...,1.000000,0.000000,1.000000,0.000000,0.990992,0.0,0.965517,1.000000,0.000000,1.000000
8,0.992795,1.0,1.000000,0.989949,0.956681,0.964209,1.000000,0.964209,1.000000,1.0,...,0.000000,0.000000,1.000000,1.000000,0.989949,0.0,0.942809,1.000000,1.000000,0.000000
9,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000


In [38]:
def PCC(mat):
    mat = np.array(pd.DataFrame(mat).fillna(0))
    NumUsers = np.size(mat, axis = 0)
    Sim = np.full((NumUsers, NumUsers), -1.0)
    #배열 a의 크기에 맞게 빈 배열을 만듭니다.
    mean = np.nanmean(np.where(mat!=0, mat, np.nan), axis=1)
    print("PCC calculation start!")
    for u in range(0, NumUsers):
        for v in range(u+1, NumUsers):
            arridx_u = np.where(mat[u, ] == 0)
            arridx_v = np.where(mat[v, ] == 0)
            arridx = np.unique(np.concatenate((arridx_u, arridx_v), axis = None))
      
            U = np.delete(mat[u, ], arridx) - mean[u]
            V = np.delete(mat[v, ], arridx) - mean[v]
            
            if(np.linalg.norm(U) == 0 or np.linalg.norm(V) == 0 ):
                Sim[u, v] = 0
            else:
                Sim[u, v] = np.dot(U, V) / (np.linalg.norm(U) * np.linalg.norm(V))

            Sim[v, u] = Sim[u, v]
    print("PCC calculation end!")
    return Sim
pd.DataFrame(PCC(matrix))

PCC calculation start!
PCC calculation end!


,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,-1.000000,0.0,-0.959214,-0.333088,-0.185921,0.671405,0.778413,0.562985,0.800168,0.0,...,1.000000e+00,-0.816497,-1.000000,0.843661,0.267928,0.0,0.693978,0.244851,0.813733,-0.136127
1,0.000000,-1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
2,-0.959214,0.0,-1.000000,-0.685146,-0.181409,1.000000,-0.569210,-0.995327,-1.000000,0.0,...,-1.000000e+00,0.000000,0.000000,0.000000,-1.000000,0.0,-1.000000,0.000000,0.000000,0.000000
3,-0.333088,0.0,-0.685146,-1.000000,0.382077,0.633808,0.000000,0.298142,-0.554700,0.0,...,9.734172e-01,0.000000,1.000000,-0.998688,1.000000,0.0,-0.575396,-0.410347,-1.000000,0.000000
4,-0.185921,0.0,-0.181409,0.382077,-1.000000,0.021657,0.857493,0.370607,-0.722296,0.0,...,1.000000e+00,0.000000,1.000000,-1.000000,0.829156,0.0,0.467888,1.000000,-1.000000,0.734433
5,0.671405,0.0,1.000000,0.633808,0.021657,-1.000000,1.000000,0.047184,-0.841191,0.0,...,0.000000e+00,0.780869,1.000000,1.000000,0.426401,0.0,0.760078,0.808143,0.000000,-1.000000
6,0.778413,0.0,-0.569210,0.000000,0.857493,1.000000,-1.000000,0.645942,-1.000000,0.0,...,1.000000e+00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
7,0.562985,0.0,-0.995327,0.298142,0.370607,0.047184,0.645942,-1.000000,-0.923782,0.0,...,1.000000e+00,0.000000,1.000000,0.000000,0.652851,0.0,0.365994,1.000000,0.000000,-1.000000
8,0.800168,0.0,-1.000000,-0.554700,-0.722296,-0.841191,-1.000000,-0.923782,-1.000000,0.0,...,0.000000e+00,0.000000,-1.000000,1.000000,-0.554700,0.0,-0.034816,-1.000000,1.000000,0.000000
9,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.0,...,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000


In [53]:
def basic_CF(mat, sim, k):
    predicted_rating = np.array([[0.0 for col in range(300)] for row in range(1000)])
    
    if(sim == 'COS'):
        Sim = COS(mat)
    else:
        Sim = PCC(mat)
    
    
    k_neighbors = np.argsort(-Sim)
    k_neighbors = np.delete(k_neighbors, np.s_[k:],1)
    
    mat = np.array(pd.DataFrame(mat).fillna(0))
    NumUsers = np.size(mat, axis = 0)
    
    for u in range(0, NumUsers):
        list_sim = Sim[u, k_neighbors[u,]]
        list_rating = mat[k_neighbors[u],].astype('float64')
        
        if np.sum(list_sim) == 0 :
            predicted_rating[u, ] = 0
        else:
            predicted_rating[u, ] = np.sum(list_sim.reshape(-1, 1) * list_rating, axis = 0) / np.sum(list_sim)
        
    return predicted_rating

pd.DataFrame(basic_CF(matrix, 'PCC', 10))

PCC calculation start!
PCC calculation end!


,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,0.800000,1.700000,1.000000,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.20000,0.0
1,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0
2,2.100000,0.000000,1.400000,0.0,0.0,0.0,0.0,0.70000,0.0,0.0,...,0.0,0.0,0.0,0.3,0.000000,0.0,0.0,0.0,0.40000,0.0
3,2.600000,0.300000,1.300000,0.3,0.0,0.0,0.0,0.40000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.2,0.80000,0.0
4,0.700000,0.000000,2.200000,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.3,0.0,0.0,1.00000,0.0
5,0.300000,2.400000,0.800000,0.0,0.1,0.0,0.0,0.00000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.300000,0.0,0.0,0.0,1.40000,0.0
6,3.000000,0.100000,0.000000,0.0,0.1,0.0,0.0,1.00000,0.0,0.0,...,0.0,0.0,0.2,0.0,0.000000,0.0,0.0,0.0,1.80000,0.0
7,1.500000,0.000000,2.400000,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,...,0.2,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.90000,0.0
8,3.800000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,...,0.2,0.0,0.0,0.0,0.200000,0.0,0.3,0.0,1.80000,0.0
9,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0


In [54]:
pd.DataFrame(basic_CF(matrix, 'COS', 10))

COS calculation start!
COS calculation end!


,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,2.2,0.0,1.1,0.0,0.0,0.0,0.0,0.4,0.0,0.3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.6,0.0
1,3.7,0.0,0.0,0.0,0.0,0.0,0.0,1.7,0.0,0.0,...,0.0,0.0,0.0,0.0,0.4,0.0,0.0,0.0,2.4,0.0
2,3.6,0.0,0.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.9,0.0,0.0,0.0,0.8,0.0
3,1.9,1.7,0.4,0.0,0.0,0.0,0.0,0.8,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.1,0.0
4,1.1,0.0,1.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3,0.0
5,2.5,0.8,0.3,0.0,0.0,0.0,0.0,0.6,0.0,0.0,...,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3,0.0
6,2.3,0.0,1.5,0.0,0.3,0.0,0.1,0.4,0.1,0.0,...,0.3,0.0,0.0,0.0,0.4,0.0,0.1,0.0,1.8,0.0
7,1.6,0.0,1.8,0.0,0.0,0.0,0.0,0.5,0.0,0.0,...,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,1.6,0.0
8,2.6,0.0,1.3,0.0,0.0,0.5,0.0,0.1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.4,0.0,0.0,0.0,1.0,0.0
9,3.7,0.0,0.0,0.0,0.0,0.0,0.0,1.7,0.0,0.0,...,0.0,0.0,0.0,0.0,0.4,0.0,0.0,0.0,2.4,0.0


In [55]:
def basic_mean(mat, sim, k):
    predicted_rating = np.array([[0.0 for col in range(300)] for row in range(1000)])
    
    mat = np.array(pd.DataFrame(mat).fillna(0))
    mean = np.nanmean(np.where(mat != 0, mat, np.nan), axis = 1)
    
    if(sim == 'COS'):
        Sim = COS(mat)
    elif(sim == 'PCC'):
        Sim = PCC(mat)
        
    k_neighbors = np.argsort(-Sim)
    k_neighbors = np.delete(k_neighbors, np.s_[k:], 1)
    
    NumUsers = np.size(mat, axis = 0)
    
    for u in range(0, NumUsers):
        list_sim = Sim[u, k_neighbors[u,]]
        list_rating = mat[k_neighbors[u,], ].astype('float64')
        list_mean = mean[k_neighbors[u, ],]
        denominator = np.sum(list_sim)
        numerator = np.sum(list_sim.reshape(-1, 1) * (list_rating - list_mean.reshape(-1, 1)), axis = 0)
        if denominator == 0 :
            predicted_rating[u, ] = 0
        else:
            predicted_rating[u, ] = mean[u] + numerator / denominator

    return predicted_rating


In [56]:
pd.DataFrame(basic_mean(matrix, 'COS', 10))

COS calculation start!
COS calculation end!


,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,2.027527,-0.172473,0.927527,-0.172473,-0.172473,-0.172473,-0.172473,0.227527,-0.172473,0.127527,...,-0.172473,-0.172473,-0.172473,-0.172473,-0.172473,-0.172473,-0.172473,-0.172473,0.427527,-0.172473
1,4.811317,1.111317,1.111317,1.111317,1.111317,1.111317,1.111317,2.811317,1.111317,1.111317,...,1.111317,1.111317,1.111317,1.111317,1.511317,1.111317,1.111317,1.111317,3.511317,1.111317
2,3.847778,0.247778,0.647778,0.247778,0.247778,0.247778,0.247778,0.247778,0.247778,0.247778,...,0.247778,0.247778,0.247778,0.247778,1.147778,0.247778,0.247778,0.247778,1.047778,0.247778
3,1.782280,1.582280,0.282280,-0.117720,-0.117720,-0.117720,-0.117720,0.682280,-0.117720,-0.117720,...,-0.117720,-0.117720,-0.117720,-0.117720,-0.117720,-0.117720,-0.117720,-0.117720,0.982280,-0.117720
4,1.504048,0.404048,2.304048,0.404048,0.404048,0.404048,0.404048,0.404048,0.404048,0.404048,...,0.404048,0.404048,0.404048,0.404048,0.404048,0.404048,0.404048,0.404048,0.704048,0.404048
5,2.408766,0.708766,0.208766,-0.091234,-0.091234,-0.091234,-0.091234,0.508766,-0.091234,-0.091234,...,0.108766,-0.091234,-0.091234,-0.091234,-0.091234,-0.091234,-0.091234,-0.091234,0.208766,-0.091234
6,3.480619,1.180619,2.680619,1.180619,1.480619,1.180619,1.280619,1.580619,1.280619,1.180619,...,1.480619,1.180619,1.180619,1.180619,1.580619,1.180619,1.280619,1.180619,2.980619,1.180619
7,1.105758,-0.494242,1.305758,-0.494242,-0.494242,-0.494242,-0.494242,0.005758,-0.494242,-0.494242,...,-0.494242,-0.494242,-0.494242,-0.494242,-0.294242,-0.494242,-0.494242,-0.494242,1.105758,-0.494242
8,2.599583,-0.000417,1.299583,-0.000417,-0.000417,0.499583,-0.000417,0.099583,-0.000417,-0.000417,...,-0.000417,-0.000417,-0.000417,-0.000417,0.399583,-0.000417,-0.000417,-0.000417,0.999583,-0.000417
9,2.811317,-0.888683,-0.888683,-0.888683,-0.888683,-0.888683,-0.888683,0.811317,-0.888683,-0.888683,...,-0.888683,-0.888683,-0.888683,-0.888683,-0.488683,-0.888683,-0.888683,-0.888683,1.511317,-0.888683


In [57]:
pd.DataFrame(basic_mean(matrix, 'PCC', 10))

PCC calculation start!
PCC calculation end!


,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,0.647596,1.547596,0.847596,-0.152404,-0.152404,-0.152404,-0.152404,-0.152404,-0.152404,-0.152404,...,-0.152404,-0.152404,-0.152404,-0.152404,-0.152404,-0.152404,-0.152404,-0.152404,0.047596,-0.152404
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,2.752698,0.652698,2.052698,0.652698,0.652698,0.652698,0.652698,1.352698,0.652698,0.652698,...,0.652698,0.652698,0.652698,0.952698,0.652698,0.652698,0.652698,0.652698,1.052698,0.652698
3,2.509848,0.209848,1.209848,0.209848,-0.090152,-0.090152,-0.090152,0.309848,-0.090152,-0.090152,...,-0.090152,-0.090152,-0.090152,-0.090152,-0.090152,-0.090152,-0.090152,0.109848,0.709848,-0.090152
4,1.093016,0.393016,2.593016,0.393016,0.393016,0.393016,0.393016,0.393016,0.393016,0.393016,...,0.393016,0.393016,0.393016,0.393016,0.393016,0.693016,0.393016,0.393016,1.393016,0.393016
5,0.239917,2.339917,0.739917,-0.060083,0.039917,-0.060083,-0.060083,-0.060083,-0.060083,-0.060083,...,-0.060083,-0.060083,-0.060083,-0.060083,0.239917,-0.060083,-0.060083,-0.060083,1.339917,-0.060083
6,4.113370,1.213370,1.113370,1.113370,1.213370,1.113370,1.113370,2.113370,1.113370,1.113370,...,1.113370,1.113370,1.313370,1.113370,1.113370,1.113370,1.113370,1.113370,2.913370,1.113370
7,0.781829,-0.718171,1.681829,-0.718171,-0.718171,-0.718171,-0.718171,-0.718171,-0.718171,-0.718171,...,-0.518171,-0.718171,-0.718171,-0.718171,-0.718171,-0.718171,-0.718171,-0.718171,0.181829,-0.718171
8,4.289567,0.489567,0.489567,0.489567,0.489567,0.489567,0.489567,0.489567,0.489567,0.489567,...,0.689567,0.489567,0.489567,0.489567,0.689567,0.489567,0.789567,0.489567,2.289567,0.489567
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [58]:
def basic_zscore(mat, sim, k):
    predicted_rating = np.array([[0.0 for col in range(300)] for row in range(1000)])
    
    mat = np.array(pd.DataFrame(mat).fillna(0))
    mean = np.nanmean(np.where(mat != 0, mat, np.nan), axis = 1)
    std = np.nanstd(np.where(mat != 0, mat, np.nan), axis = 1)
    
    if(sim == 'COS'):
        Sim = COS(mat)
    elif(sim == 'PCC'):
        Sim = PCC(mat)
        
    k_neighbors = np.argsort(-Sim)
    k_neighbors = np.delete(k_neighbors, np.s_[k:], 1)
    
    NumUsers = np.size(mat, axis = 0)
    
    for u in range(0, NumUsers):
        list_sim = Sim[u, k_neighbors[u,]]#한줄
        list_rating = mat[k_neighbors[u,], ].astype('float64')#전체
        list_mean = mean[k_neighbors[u, ],]#한줄
        list_std = std[k_neighbors[u, ], ]#한줄
        #print(list_std.reshape(-1,1))
        denominator = np.sum(list_sim)
        
        arridx = np.where(list_std == 0)
        
        list_std[arridx] = 1
        list_sim[arridx] = 0
        
        #print(list_std)
        
        numerator = np.sum(list_sim.reshape(-1, 1) * ((list_rating - list_mean.reshape(-1, 1)) / list_std.reshape(-1,1)), axis = 0)
        
        if denominator == 0 :
            predicted_rating[u, ] = 0
        else:
            predicted_rating[u, ] = mean[u] + std[u] * numerator / denominator

    return predicted_rating

In [59]:
pd.DataFrame(basic_zscore(matrix, 'COS', 10))

COS calculation start!
COS calculation end!


,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,1.038509,0.067678,2.260480,0.067678,0.067678,0.067678,0.067678,0.467678,0.067678,0.887566,...,0.067678,0.067678,0.067678,0.067678,0.067678,0.067678,0.067678,0.067678,1.027675,0.067678
1,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,...,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000
2,3.780382,2.083786,2.495262,2.083786,2.083786,2.083786,2.083786,2.083786,2.083786,2.083786,...,2.083786,2.083786,2.083786,2.083786,2.431722,2.083786,2.083786,2.083786,2.779833,2.083786
3,1.224528,0.726925,-0.993856,-1.439217,-1.439217,-1.439217,-1.439217,-0.123354,-1.439217,-1.439217,...,-1.439217,-1.439217,-1.439217,-1.439217,-1.439217,-1.439217,-1.439217,-1.439217,0.139850,-1.439217
4,0.774640,0.539393,3.064315,0.539393,0.539393,0.539393,0.539393,0.539393,0.539393,0.539393,...,0.539393,0.539393,0.539393,0.539393,0.539393,0.539393,0.539393,0.539393,1.139393,0.539393
5,2.466274,0.062512,-0.733920,-0.872850,-0.872850,-0.872850,-0.872850,-0.141605,-0.872850,-0.872850,...,-0.505275,-0.872850,-0.872850,-0.872850,-0.872850,-0.872850,-0.872850,-0.872850,-0.573832,-0.872850
6,3.841048,2.642689,3.584468,2.642689,2.850059,2.642689,2.693869,2.933731,2.693869,2.642689,...,2.850059,2.642689,2.642689,2.642689,2.851401,2.642689,2.693869,2.642689,3.680681,2.642689
7,0.757441,-0.708424,1.315809,-0.708424,-0.708424,-0.708424,-0.708424,0.024508,-0.708424,-0.708424,...,-0.708424,-0.708424,-0.708424,-0.708424,-0.526606,-0.708424,-0.708424,-0.708424,0.987737,-0.708424
8,2.912457,1.708916,2.700118,1.708916,1.708916,2.067787,1.708916,1.764520,1.708916,1.708916,...,1.708916,1.708916,1.708916,1.708916,1.996013,1.708916,1.708916,1.708916,2.380294,1.708916
9,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,...,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000


In [60]:
pd.DataFrame(basic_zscore(matrix, 'PCC', 10))

PCC calculation start!
PCC calculation end!


,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,-0.208675,0.989355,0.373832,-1.117535,-1.117535,-1.117535,-1.117535,-1.117535,-1.117535,-1.117535,...,-1.117535,-1.117535,-1.117535,-1.117535,-1.117535,-1.117535,-1.117535,-1.117535,-0.769686,-1.117535
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,3.052478,0.483554,1.412570,0.483554,0.483554,0.483554,0.483554,1.019905,0.483554,0.483554,...,0.483554,0.483554,0.483554,0.626377,0.483554,0.483554,0.483554,0.483554,0.877082,0.483554
3,2.700395,-1.097462,-0.262535,-1.144707,-1.454546,-1.454546,-1.454546,-1.041428,-1.454546,-1.454546,...,-1.454546,-1.454546,-1.454546,-1.454546,-1.454546,-1.454546,-1.454546,-1.247987,-0.734912,-1.454546
4,0.059396,-1.016938,2.112022,-1.016938,-1.016938,-1.016938,-1.016938,-1.016938,-1.016938,-1.016938,...,-1.016938,-1.016938,-1.016938,-1.016938,-1.016938,-0.393399,-1.016938,-1.016938,0.353258,-1.016938
5,0.504505,2.356280,0.910671,0.104939,0.148347,0.104939,0.104939,0.104939,0.104939,0.104939,...,0.104939,0.104939,0.104939,0.104939,0.235163,0.104939,0.104939,0.104939,1.628599,0.104939
6,4.172961,1.887113,1.834840,1.834840,1.887113,1.834840,1.834840,2.572011,1.834840,1.834840,...,1.834840,1.834840,1.939386,1.834840,1.834840,1.834840,1.834840,1.834840,3.127751,1.834840
7,0.372758,-1.835421,1.017762,-1.835421,-1.835421,-1.835421,-1.835421,-1.835421,-1.835421,-1.835421,...,-1.442088,-1.835421,-1.835421,-1.835421,-1.835421,-1.835421,-1.835421,-1.835421,-0.803785,-1.835421
8,4.285145,0.416950,0.416950,0.416950,0.416950,0.416950,0.416950,0.416950,0.416950,0.416950,...,0.785732,0.416950,0.416950,0.416950,0.568710,0.416950,0.644589,0.416950,2.048074,0.416950
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [69]:
def basic_baseline(mat, sim, k):
    predicted_rating = np.array([[0.0 for col in range(300)] for row in range(1000)])
    
    mat = np.array(pd.DataFrame(mat).fillna(0))
    user_mean = np.nanmean(np.where(mat != 0, mat, np.nan), axis = 1)
    item_mean = np.nanmean(np.where(mat != 0, mat, np.nan), axis = 0) #아이템 평균 배열
    
    all_avg = np.sum(mat)/(mat>0).sum() #모든 rating 평균
    
    if(sim == 'COS'):
        Sim = COS(mat)
    elif(sim == 'PCC'):
        Sim = PCC(mat)
        
    k_neighbors = np.argsort(-Sim)
    k_neighbors = np.delete(k_neighbors, np.s_[k:], 1)
    
    

    NumUsers = np.size(mat, axis = 0)
    
    baseline = user_mean.reshape(-1, 1) + item_mean - all_avg
    
    for u in range(0, NumUsers):
        list_sim = Sim[u, k_neighbors[u,]]
        list_rating = mat[k_neighbors[u,], ].astype('float64')    
        list_mean = user_mean[k_neighbors[u, ],]
        denominator = np.sum(list_sim)
        
        numerator = np.sum(list_sim.reshape(-1, 1) * (list_rating - baseline[u,]), axis = 0)
        
        if denominator == 0:
            predicted_rating[u, ] = 0
        else:
            predicted_rating[u, ] = (baseline[u,].reshape(1,-1) + (numerator / denominator).reshape(1,-1))

    return predicted_rating

In [70]:
pd.DataFrame(basic_baseline(matrix, 'COS', 10))

COS calculation start!
COS calculation end!


,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,2.200000e+00,-4.440892e-16,1.100000e+00,0.000000e+00,0.000000e+00,-4.440892e-16,-4.440892e-16,4.000000e-01,-4.440892e-16,3.000000e-01,...,-4.440892e-16,-4.440892e-16,-4.440892e-16,-8.881784e-16,-4.440892e-16,-4.440892e-16,0.000000e+00,0.000000e+00,6.000000e-01,-4.440892e-16
1,3.700000e+00,0.000000e+00,8.881784e-16,-4.440892e-16,8.881784e-16,0.000000e+00,0.000000e+00,1.700000e+00,4.440892e-16,0.000000e+00,...,-4.440892e-16,-8.881784e-16,-8.881784e-16,-8.881784e-16,4.000000e-01,0.000000e+00,0.000000e+00,-4.440892e-16,2.400000e+00,-4.440892e-16
2,3.600000e+00,8.881784e-16,4.000000e-01,0.000000e+00,-4.440892e-16,-4.440892e-16,-4.440892e-16,0.000000e+00,0.000000e+00,-4.440892e-16,...,-4.440892e-16,-8.881784e-16,-4.440892e-16,8.881784e-16,9.000000e-01,4.440892e-16,0.000000e+00,0.000000e+00,8.000000e-01,-4.440892e-16
3,1.900000e+00,1.700000e+00,4.000000e-01,4.440892e-16,0.000000e+00,-8.881784e-16,0.000000e+00,8.000000e-01,0.000000e+00,-8.881784e-16,...,-4.440892e-16,-8.881784e-16,-4.440892e-16,-4.440892e-16,-4.440892e-16,-8.881784e-16,4.440892e-16,0.000000e+00,1.100000e+00,-4.440892e-16
4,1.100000e+00,8.881784e-16,1.900000e+00,-4.440892e-16,0.000000e+00,-4.440892e-16,0.000000e+00,-4.440892e-16,-4.440892e-16,-4.440892e-16,...,0.000000e+00,-4.440892e-16,-4.440892e-16,-4.440892e-16,4.440892e-16,0.000000e+00,-4.440892e-16,4.440892e-16,3.000000e-01,0.000000e+00
5,2.500000e+00,8.000000e-01,3.000000e-01,0.000000e+00,0.000000e+00,-4.440892e-16,-4.440892e-16,6.000000e-01,0.000000e+00,-4.440892e-16,...,2.000000e-01,-4.440892e-16,-4.440892e-16,-4.440892e-16,-4.440892e-16,-4.440892e-16,0.000000e+00,0.000000e+00,3.000000e-01,-4.440892e-16
6,2.300000e+00,0.000000e+00,1.500000e+00,-4.440892e-16,3.000000e-01,-4.440892e-16,1.000000e-01,4.000000e-01,1.000000e-01,-4.440892e-16,...,3.000000e-01,-8.881784e-16,-8.881784e-16,-8.881784e-16,4.000000e-01,-4.440892e-16,1.000000e-01,-4.440892e-16,1.800000e+00,-4.440892e-16
7,1.600000e+00,-4.440892e-16,1.800000e+00,-6.661338e-16,0.000000e+00,-8.881784e-16,0.000000e+00,5.000000e-01,0.000000e+00,-8.881784e-16,...,0.000000e+00,0.000000e+00,0.000000e+00,-8.881784e-16,2.000000e-01,-4.440892e-16,-6.661338e-16,0.000000e+00,1.600000e+00,0.000000e+00
8,2.600000e+00,-8.881784e-16,1.300000e+00,0.000000e+00,-4.440892e-16,5.000000e-01,-4.440892e-16,1.000000e-01,0.000000e+00,4.440892e-16,...,-4.440892e-16,4.440892e-16,-4.440892e-16,0.000000e+00,4.000000e-01,-4.440892e-16,0.000000e+00,0.000000e+00,1.000000e+00,-4.440892e-16
9,3.700000e+00,4.440892e-16,4.440892e-16,0.000000e+00,-4.440892e-16,0.000000e+00,0.000000e+00,1.700000e+00,-2.220446e-16,0.000000e+00,...,0.000000e+00,-4.440892e-16,4.440892e-16,0.000000e+00,4.000000e-01,0.000000e+00,0.000000e+00,0.000000e+00,2.400000e+00,0.000000e+00


In [71]:
pd.DataFrame(basic_baseline(matrix, 'PCC', 10))

PCC calculation start!
PCC calculation end!


,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,8.000000e-01,1.700000e+00,1.000000e+00,0.000000e+00,0.000000e+00,-4.440892e-16,0.000000e+00,0.000000e+00,0.000000e+00,-4.440892e-16,...,-4.440892e-16,-4.440892e-16,0.000000e+00,-4.440892e-16,-4.440892e-16,-4.440892e-16,0.000000e+00,0.000000e+00,2.000000e-01,-4.440892e-16
1,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
2,2.100000e+00,8.881784e-16,1.400000e+00,0.000000e+00,-4.440892e-16,-4.440892e-16,-4.440892e-16,7.000000e-01,0.000000e+00,-4.440892e-16,...,-4.440892e-16,-8.881784e-16,-4.440892e-16,3.000000e-01,-8.881784e-16,4.440892e-16,0.000000e+00,0.000000e+00,4.000000e-01,-4.440892e-16
3,2.600000e+00,3.000000e-01,1.300000e+00,3.000000e-01,0.000000e+00,-4.440892e-16,0.000000e+00,4.000000e-01,0.000000e+00,-4.440892e-16,...,-4.440892e-16,-8.881784e-16,-4.440892e-16,-4.440892e-16,-4.440892e-16,-4.440892e-16,4.440892e-16,2.000000e-01,8.000000e-01,-4.440892e-16
4,7.000000e-01,8.881784e-16,2.200000e+00,-4.440892e-16,0.000000e+00,-4.440892e-16,0.000000e+00,-4.440892e-16,-4.440892e-16,-4.440892e-16,...,0.000000e+00,-4.440892e-16,-4.440892e-16,-4.440892e-16,4.440892e-16,3.000000e-01,-4.440892e-16,4.440892e-16,1.000000e+00,0.000000e+00
5,3.000000e-01,2.400000e+00,8.000000e-01,0.000000e+00,1.000000e-01,-4.440892e-16,-4.440892e-16,-4.440892e-16,0.000000e+00,-4.440892e-16,...,-4.440892e-16,-4.440892e-16,-4.440892e-16,-4.440892e-16,3.000000e-01,-4.440892e-16,0.000000e+00,0.000000e+00,1.400000e+00,-4.440892e-16
6,3.000000e+00,1.000000e-01,8.881784e-16,-4.440892e-16,1.000000e-01,-4.440892e-16,4.440892e-16,1.000000e+00,0.000000e+00,-4.440892e-16,...,-4.440892e-16,-8.881784e-16,2.000000e-01,-8.881784e-16,-8.881784e-16,-4.440892e-16,-4.440892e-16,-4.440892e-16,1.800000e+00,-4.440892e-16
7,1.500000e+00,-4.440892e-16,2.400000e+00,0.000000e+00,0.000000e+00,-4.440892e-16,0.000000e+00,0.000000e+00,2.220446e-16,-4.440892e-16,...,2.000000e-01,0.000000e+00,0.000000e+00,-4.440892e-16,-4.440892e-16,-4.440892e-16,0.000000e+00,0.000000e+00,9.000000e-01,0.000000e+00
8,3.800000e+00,0.000000e+00,0.000000e+00,4.440892e-16,-4.440892e-16,4.440892e-16,-4.440892e-16,0.000000e+00,4.440892e-16,4.440892e-16,...,2.000000e-01,4.440892e-16,0.000000e+00,0.000000e+00,2.000000e-01,-4.440892e-16,3.000000e-01,4.440892e-16,1.800000e+00,-4.440892e-16
9,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00


In [ ]:
A = np.array([[4,1,1,4,2,3,5,0,4,0],
             [0,4,2,0,3,2,5,0,4,3],
             [2,0,4,5,2,0,1,3,4,2],
             [1,4,0,1,4,5,0,3,1,2],
             [1,2,3,4,0,3,4,4,2,5]])